[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2020CSharp/main?urlpath=lab%2Ftree%2FDay17.ipynb)

# --- Day 17: Conway Cubes ---

In [1]:
using System.IO;
using System.Numerics;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Made use of the `Vector3` struct from the [`System.Numerics`](https://docs.microsoft.com/en-us/dotnet/api/system.numerics) namespace.

In [2]:
var input = File.ReadAllLines(@"input/17.txt").Select(line => line.ToCharArray()).ToArray();

Solved this in two parts. The first part will expand the cube in all directions with the inactive (`"."`) state.

In [3]:
Dictionary<Vector3, char> ExpandCubes(Dictionary<Vector3, char> cubes)
{
    var newCubes = new Dictionary<Vector3, char>(cubes);
    
    var directions = Enumerable.Range(-1, 3).Select(
        x => Enumerable.Range(-1, 3).Select(
            y => Enumerable.Range(-1, 3).Select(z => new Vector3(x, y, z))
        ).SelectMany(v => v)
    ).SelectMany(v => v);
    
    foreach (var cube in cubes)
    {
        foreach (var d in directions)
        {
            var newVector = cube.Key + d;
            newCubes[newVector] = cubes.ContainsKey(newVector) ? cubes[newVector] : '.';
        }
    }
    
    return newCubes;
}

The second part will perform one cycle of the cubes. Go through each cube and count the number of active neighbors. Then set the value accordingly.

In [4]:
Dictionary<Vector3, char> CycleCubes(Dictionary<Vector3, char> cubes)
{
    var newCubes = new Dictionary<Vector3, char>(cubes);
    var directions = Enumerable.Range(-1, 3).Select(
        x => Enumerable.Range(-1, 3).Select(
            y => Enumerable.Range(-1, 3).Select(z => new Vector3(x, y, z))
        ).SelectMany(v => v)
    ).SelectMany(v => v).Where(v => v != new Vector3(0, 0, 0));
    
    foreach (var cube in cubes)
    {
        var activeNeighbors = directions
            .Where(d => cubes.ContainsKey(cube.Key + d))
            .Select(d => cubes[cube.Key + d])
            .Where(c => c == '#')
            .Count();
        
        if (cube.Value == '#')
        {
            newCubes[cube.Key] = activeNeighbors == 2 || activeNeighbors == 3 ? '#' : '.';
        }
        else if (cube.Value == '.')
        {
            newCubes[cube.Key] = activeNeighbors == 3 ? '#' : '.';
        }
    }
    return newCubes;
}

This is a helper function for debugging. It will print out the cubes in the same format as the example.

In [5]:
void PrintCubes(Dictionary<Vector3, char> cubes)
{
    foreach (var z in cubes.GroupBy(kv => kv.Key.Z).OrderBy(grp => grp.Key))
    {
        var layer = String.Join(
            "\n", 
            z.GroupBy(kv => kv.Key.Y)
                .OrderBy(grp => grp.Key)
                .Select(
                    grp => new string(
                        grp.OrderBy(kv => kv.Key.X)
                            .Select(kv => kv.Value)
                            .ToArray()
                    )
                )
        );
        Console.WriteLine($"z={z.Key}");
        Console.WriteLine(layer);
        Console.WriteLine();
    }
}

Covert the puzzle input to a dictionary where the key is the vector and the value is the state (active / inactive). Run six cycles. Then count the number of active cubes.

In [6]:
#!time
var cubes = input
    .Select((row, y) => row.Select((cube, x) => (Vector: new Vector3(x, y, 0), Value: cube)))
    .SelectMany(v => v)
    .ToDictionary(t => t.Vector, t => t.Value);

for (var i = 0; i < 6; i++)
{
    cubes = CycleCubes(ExpandCubes(cubes));
}
cubes.Values.Where(c => c == '#').Count()

293

Wall time: 392.6857ms

# --- Part Two ---

Copy/pasted from above, but with the added dimension. Thanksfully the `System.Numerics` namespace had a `Vector4` struct, too.

In [7]:
Dictionary<Vector4, char> ExpandCubes2(Dictionary<Vector4, char> cubes)
{
    var newCubes = new Dictionary<Vector4, char>(cubes);
    
    var directions = Enumerable.Range(-1, 3).Select(
        x => Enumerable.Range(-1, 3).Select(
            y => Enumerable.Range(-1, 3).Select(
                z => Enumerable.Range(-1, 3).Select(
                    w => new Vector4(w, x, y, z)
                )
            ).SelectMany(v => v)
        ).SelectMany(v => v)
    ).SelectMany(v => v);
    
    foreach (var cube in cubes)
    {
        foreach (var d in directions)
        {
            var newVector = cube.Key + d;
            newCubes[newVector] = cubes.ContainsKey(newVector) ? cubes[newVector] : '.';
        }
    }
    
    return newCubes;
}

In [8]:
Dictionary<Vector4, char> CycleCubes2(Dictionary<Vector4, char> cubes)
{
    var newCubes = new Dictionary<Vector4, char>(cubes);
    var directions = Enumerable.Range(-1, 3).Select(
        x => Enumerable.Range(-1, 3).Select(
            y => Enumerable.Range(-1, 3).Select(
                z => Enumerable.Range(-1, 3).Select(
                    w => new Vector4(w, x, y, z)
                )
            ).SelectMany(v => v)
        ).SelectMany(v => v)
    ).SelectMany(v => v).Where(v => v != new Vector4(0, 0, 0, 0));
    
    foreach (var cube in cubes)
    {
        var activeNeighbors = directions
            .Where(d => cubes.ContainsKey(cube.Key + d))
            .Select(d => cubes[cube.Key + d])
            .Where(c => c == '#')
            .Count();
        
        if (cube.Value == '#')
        {
            newCubes[cube.Key] = activeNeighbors == 2 || activeNeighbors == 3 ? '#' : '.';
        }
        else if (cube.Value == '.')
        {
            newCubes[cube.Key] = activeNeighbors == 3 ? '#' : '.';
        }
    }
    return newCubes;
}

In [9]:
#!time
var cubes2 = input
    .Select((row, y) => row.Select((cube, x) => (Vector: new Vector4(0, x, y, 0), Value: cube)))
    .SelectMany(v => v)
    .ToDictionary(t => t.Vector, t => t.Value);

for (var i = 0; i < 6; i++)
{
    cubes2 = CycleCubes2(ExpandCubes2(cubes2));
}

cubes2.Values.Where(c => c == '#').Count()

1816

Wall time: 3415.5301ms